# Implementation of a convolutional RBM using Theano

This notebook implements an efficient cRBM with various training procedures and different layers.

## Part 0: Importing theano, Biopython and all the other useful tools

In [2]:
import theano
import theano.tensor as T
import theano.tensor.nnet.conv as conv


import numpy as np
import Bio.SeqIO as sio
import Bio.motifs.matrix as mat
from Bio.Alphabet import IUPAC
from Bio.Seq import Seq
from Bio import motifs

import random
import time

ERROR (theano.sandbox.cuda): Failed to compile cuda_ndarray.cu: libcublas.so.7.0: cannot open shared object file: No such file or directory
ERROR:theano.sandbox.cuda:Failed to compile cuda_ndarray.cu: libcublas.so.7.0: cannot open shared object file: No such file or directory
WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: cuda unavilable)


## Part 1: Getting and transforming the data

In [3]:
def readFASTASequencesFromFile (filename):
    dhsSequences = []
    for dhs in sio.parse(open(filename), 'fasta', IUPAC.unambiguous_dna):
        dhsSequences.append(dhs.seq)
    return dhsSequences

def readPWMsFromFile (filename):
    matrices = []
    for mat in motifs.parse(open(filename), 'jaspar'):
        matrices.append(mat.pwm)
    return matrices

### Read in FASTA sequences
These sequences are DNase1-hypersensitivity sites where each sample represents one DHS.
They were all already brought to the same dimensionality.

In [15]:
allSeqs = readFASTASequencesFromFile('../data/wgEncodeAwgDnaseUwAg10803UniPk.fa')

In [5]:
pwms = readPWMsFromFile('../data/jaspar_matrices.txt')

### Create a test set
Since working with these huge amounts of sequences is very time consuming, just start with a selection of them.

In [16]:
test_set = [allSeqs[random.randrange(0,len(allSeqs))] for i in range(1000)]
print "Length of test set: "  + str(len(test_set))

Length of test set: 1000


### Convert test set sequences to matrices we can work with
Each sequence is converted to a matrix of dimensionality (2 x 4 x N_v), representing:
* the two strands (first dimension)
* the four letters of the DNA alphabet (second dimension)
* the sequence represented by 1 (letter is present at this position) or 0 (letter is not present) (third dimension)

In [17]:
def getIntToLetter (letter):
    if letter == 'A' or letter == 'a':
        return 0
    elif letter == 'C' or letter == 'c':
        return 1
    elif letter == 'G' or letter == 'g':
        return 2
    elif letter == 'T' or letter == 't':
        return 3
    else:
        print "ERROR. LETTER " + letter + " DOES NOT EXIST!"
        return -1

def getMatrixFromSeq (seq):
    m = len(seq.alphabet.letters)
    n = len(seq)
    result = np.zeros((2, m, n))
    revSeq = seq.reverse_complement()
    for i in range(len(seq)):
        result[0,getIntToLetter(seq[i]),i] = 1
        result[1,getIntToLetter(revSeq[i]),i] = 1
    return result

In [18]:
start = time.time()
dataMat = np.array([getMatrixFromSeq(t) for t in test_set])
print "Conversion of sequence to matrix for the test set in (in ms): " + str((time.time()-start)*1000)

Conversion of sequence to matrix for the test set in (in ms): 298.466920853


## Part 2: Implementing the cRBM
We now have all the tools we need to actually implement the cRBM.
Some notes on constraints and behavior of our learning algorithm:
* It uses Theano to do most (if not all) of the work. That means it's ways faster on a GPU
* It expects kernels (filters/motifs) to be of the same length. While this choice was made due to performance issues, it's not a problem because we expect the algorithm to combine multiple motifs if nessessary.
* The sequences (DHSes) are also expected to be of the same size

Now, finally some hints on usage:

Blah, blah, blah

In [103]:
"""
This class implements a cRBM for sequence analysis.
It does perform efficient forward and backward pass of any given DNA sequence.
It implements softmax and sigmoid functions as activation and performs probabilistic max pooling
after the convolution step.
So this class is basically a two layer network with a convolution layer and a pooling layer on top
of that.
The learning procedure uses contrastive divergence (CD) with a variable amount of steps.
"""
class ConvRBM:
    
    """
    Initialize the cRBM. The parameters here are global params that should not change
    during the execution of training or testing and characterize the network.
    
    Parameters:
    _motifLength:    How long are the motifs (position weight matrices PWM). This
                     This is equivalent to ask what the number of k-mers is.
                     The current approach only deals with one fixed motif length.
                     
    _numMotifs:      How many motifs are applied to the sequence, that is how many
                     hidden units does the network have. Each hidden unit consists
                     of a vector of size (sequenceLength-motifLength+1)
                     
    _poolingFactor:  How many units from the hidden layer are pooled together.
                     Note that the number has to divide evenly to the length of
                     the hidden units, that is:
                     mod(sequenceLength-motifLength+1, poolingFactor) == 0
                     (1 = equivalent to sigmoid activation)
                     
    _alphabet:       Biopython uses alphabets for sequences to do sanity checks.
                     However, all of the code is written for DNA sequences and even
                     though in theory there should be no difference between that
                     and other alphabets, Biopython may have trouble with the convolution.
    """
    def __init__ (self, _motifLength, _numMotifs, _learningRate=0.1, _poolingFactor=1, _alphabet=IUPAC.unambiguous_dna):
        # parameters for the motifs
        self.motifLength = _motifLength
        self.numMotifs = _numMotifs
        self.motifs = np.random.rand(_numMotifs, 2, 4, _motifLength)
        self.alphabet = _alphabet
        
        # cRBM parameters
        self.bias = np.random.rand(self.numMotifs)
        self.c = random.random()
        self.poolingFactor = _poolingFactor
        self.learningRate = _learningRate
        
        # infrastructural parameters
        self.rng = np.random.RandomState()
        
        
    """
    Calculate the forward pass for any given set of sequences, that is calculate P(H | V).
    This method applies convolution of all filters to all sequences in the set, using theano.
    It also looks on both strands for matches and returns the hidden activation layer.
    
    Parameters:
    data:            The DNA sequences to calculate the forward pass on. The data is a 4D matrix
                     with dimensionality: (N_batch x numOfStrands(2) x numOfLetters(4) x N_v)
                     
    Return:
    The function returns the hidden activation layer as numpy matrix.
    That matrix has dimensionality N_batch x K x 1 x N_h where
    K is the number of kernels (motifs/PWMs) that are applied and
    N_h is the length of the hidden layer (convolution is of type 'valid')
    
    Note that the strandness is lost during this process because it's added up in the hidden layer.
    """
    def forwardBatch (self, data):
        # create 4D tensor for theano (BatchSize x K x 2*numOfLetters x lenOfSeqs)
        D = T.tensor4('data')
        K = T.tensor4('kernels')
        out = conv.conv2d(D,K)
        f = theano.function([D,K], out, allow_input_downcast=True)
        return f(data, self.motifs)



    """
    Calculates the backward pass on the data, that is P(V | H).
    It does so by performing convolution of the hidden layer and the motifs for each kernel using theano.
    
    Parameters:
    hidden layer:   The layer that was previously computed by the forward pass.
    
    Return:
    A matrix of dimensionality: N_batch x K x numOfLetters(4) x N_v
    The result can be interpreted as the probability for each position to have a specific letter
    once it is summed over all K (sum over second dimension).
    That means, the combination of summing over K and applying softmax can be interpreted as P(V | H).
    """
    def backwardBatch (self, hiddenActivation):
        # theano convolution call
        H = T.tensor4('hidden')
        K = T.tensor4('kernels')
        K_star = K.dimshuffle(1, 0, 2, 3)[:,:,::-1,::-1]
        C = conv.conv2d(H, K_star, border_mode='full')
        out = T.sum(C, axis=1) # sum over all K
        f = theano.function([H,K], out, allow_input_downcast=True)

        return f(hiddenActivation, self.motifs)



    """
    Set new kernels when you don't wish to start with random ones. Can be used if prior knowledge about
    the structure of the sequences is present.
    
    Parameters:
    customKernels:  A matrix of shape (K x numOfStrands(2) x numOfLetters(4) x num-of-k-mers) containing the
                    new kernels to work with.
                    
    Return:
    Nothing.
    """
    def setCustomKernels (self, customKernels):
        if len(customKernels.shape) != 4:
            print "New motifs must be a 4D matrix with dims: (K x numOfStrands(2) x numOfLetters(4) x numOfKMers)"
            return
        
        self.numMotifs = customKernels.shape[0]
        self.motifLength = customKernels.shape[3]
        print "New motifs set. # Motifs: " + str(self.numMotifs) + " K-mer-Length: " + str(self.motifLength)
        self.motifs = customKernels



    """
    Calculate the gradient for a given data matrix and samples from the hidden layer.
    The gradient is computed using theano and convolution.
    
    Parameters:
    hiddenSample:   A matrix of shape (N_batch x K x numOfLetters(4) x N_v) containing samples
                    from the prob. distribution of the hidden layer.
                    Note that this matrix has the same structure as the result from forwardBatch().
                    
    data:           A matrix of shape (N_batch x numOfStrands(2) x numOfLetters(4) x N_v) containing
                    the data. Note that this matrix is usually the same as the matrix passed to forwardBatch().
                    
    Return:
    A matrix containing the gradient for all kernels/motifs/pwms.
    This matrix is of shape (N_batch x K x 1 x num-of-k-mers)
    """
    def gradient (self, hiddenSample, data):
        H = T.tensor4('hidden')
        S = T.tensor4('sample')
        H_reshaped = H.dimshuffle(1, 0, 2, 3)
        out = conv.conv2d(S, H_reshaped)
        f = theano.function([H,S], out, allow_input_downcast=True)

        return f(np.tile(np.mean(hiddenSample, axis=0), [2,1,1,1]), np.tile(data, [1,1,1,1]))



    """
    The training algorithm for cRBMs. This uses the forward and backward pass
    to collect the statistics for them.
    """
    def miniBatchTraining (self, sequences, batchSize, cd_value):
        
        # first, some vars that we need all the time
        sequenceLength = len(sequences[0])
        hiddenUnitLength = sequenceLength - self.motifLength + 1
        
        # first, compute expected value of the data
        # that means computing forward pass for all datapoints and computing the gradient
        hiddenActivation = np.zeros((2*self.numMotifs, hiddenUnitLength))
        derivatives = np.zeros((self.motifLength, self.numMotifs))
        for seq in sequences:
            hiddenActivation = self.forwardPass(seq)
            
            # calculate the gradients of the data
            # In order to do that, we have to convolve the DNA and hidden layer
            # This can be done by expressing DNA as four different convolutions (for each letter)
            # We would need to represent the DNA as matrix of 4 x lengthOfDHS
            # Example:
            # -------
            # ACGTGGGG
            # 10000000
            # 01000000
            # 00101111
            # 00010000
            # Then, we can apply 4 convolutions to the sequence
            for k in range(self.numMotifs):
                derivatives[k,:] += np.convolve( hiddenActivation[k,:])
            
        print derivatives
        # Now, sample from this distribution and collect the model's statistics
        # Do that by sampling from the data statistics, recompute the model's statistics
        # and so on until reaching the stationary distribution (in case of CD, do it once)
        hidden_model = hiddenActivation
        for it in range(cd_value):
            sample = self._sampleFromHidden(hidden_model)
            visible = self.backwardPass(sample)
            hidden_model = self.forwardPass(visible)

        print hidden_model.shape
        
        return self.learningRate * (hiddenActivation - hidden_model)
   


    """
    For sure deprecated.
    """
    def _probMaxPooling (self, h_k):

        # first of all some easy definitions
        l = h_k.shape[1]
        numOfGroups = l/self.poolingFactor
        P = np.zeros((2, l))

        # exponent of everything
        ex = np.exp(h_k)
        
        # reshape s.t. each group forms one row
        newDim = (numOfGroups, -1)
        reordered = np.append(ex[0].reshape(newDim), ex[1].reshape(newDim), 1)
        #print "Shape of reordered: " + str(reordered.shape)
        # calculate denominators (sum of all rows)
        denoms = np.sum(reordered, 1) + 1 # denoms for all groups (add 1 to have log. unit)
        res = np.argmax(reordered.T / denoms, 0)

        # calculate the actual values of the pooling layer P
        for group in range(numOfGroups):
            if reordered[group,res[group]] > 1: # check if really any element from P should be active
                # we don't care about strands so just set res = res/2 for the index
                idx = group * self.poolingFactor + int(res[group]/2)
                P[res[group] % 2, idx] = reordered[group,res[group]] / denoms[group]
        return P



    """
    Maybe not working...
    """
    def _sampleFromHidden (self, hiddenActivation):
        boolean_mat = hiddenActivation > self.rng.random_sample(hiddenActivation.shape)
        return boolean_mat.astype(int)



    """
    Calculate the sigmoid activation for the hidden layer.
    """
    def sigmoid(self, _H):
        return (1.0 / (1.0 + np.exp(-_H)))



    def softmax (self, V):
        # insert ones (that is the 5th component 1/1+e^x)
        #withOnes = np.insert(V, 4, np.ones(V.shape[2]), 1)
        exp = np.exp(V) # exponentiate it all
        denominator = np.sum(exp, axis=1) # this the sum of all the exp(-x)
        # now expand the denominator such that the division works
        denominator = denominator[:,np.newaxis,:].repeat(exp.shape[1], axis=1)
        div = exp / denominator
        return div


    """
    Maybe deprecated.
    """
    def _getLetterToInt (self, num):
        if num == 0:
            return 'A'
        elif num == 1:
            return 'C'
        elif num == 2:
            return 'G'
        elif num == 3:
            return 'T'
        else:
            print 'ERROR: Num ' + str(num) + " not a valid char in DNA alphabet"
            return -1



    """
    Maybe deprecated.
    """
    def _getDNASeqFromNumericals (self, seq):
        dna_seq = []
        for num in range(seq.shape[0]):
            dna_seq.append(self._getLetterToInt(seq[num]))
        return Seq("".join(dna_seq), alphabet=self.alphabet)

### Verify that the algorithm is doing something meaningful
We can now apply the learning algorithm on a toy sequence to see what happens.
Important questions to ask:
* Does the hidden layer somehow make sense?
* What does the sigmoid function do with it?
* How does the *reconstruction* look like?
* Is the softmax doing the right thing?

In [113]:
# Construct a kernel matrix, looking for ACGT and GGGG
kernel1 = np.tile(np.eye(4), [2,1,1]) # the kernel will only look for ACGT
kernel2 = np.tile(np.array([[0,0,0,0],[0,0,0,0],[1,1,1,1],[0,0,0,0]]), [2,1,1]) # this one looks for GGGG
kernel = np.array([kernel1])

# create toy sequences where we might be able to see what happens
randSeq1 = getMatrixFromSeq(Seq("ACGTGGGG", IUPAC.unambiguous_dna))
randSeq2 = getMatrixFromSeq(Seq("ACGTACGT", IUPAC.unambiguous_dna))
#data = np.array([randSeq1, randSeq2])
data = np.array([randSeq1])
# initialize the learner and insert our predefined kernels
cRBM = ConvRBM (4, 2)
cRBM.setCustomKernels(kernel)

np.set_printoptions(precision=2, suppress=True)
res = cRBM.sigmoid(cRBM.forwardBatch(data))
print "Result from forward pass (that is: P(H | V))"
print "Shape: -> " + str(res.shape)
#print res
vis = cRBM.softmax(cRBM.backwardBatch(res))
print "Result from backward pass (that is: P(V | H))"
print "Shape: -> " + str(vis.shape)
#print vis
grad = cRBM.gradient(res, data)

New motifs set. # Motifs: 1 K-mer-Length: 4
Result from forward pass (that is: P(H | V))
Shape: -> (1, 1, 1, 5)
Result from backward pass (that is: P(V | H))
Shape: -> (1, 4, 8)


array([[[ 0.71,  0.32,  0.32,  0.2 ,  0.34,  0.05,  0.07,  0.1 ],
        [ 0.1 ,  0.54,  0.23,  0.27,  0.2 ,  0.4 ,  0.07,  0.1 ],
        [ 0.1 ,  0.07,  0.4 ,  0.2 ,  0.27,  0.23,  0.54,  0.1 ],
        [ 0.1 ,  0.07,  0.05,  0.34,  0.2 ,  0.32,  0.32,  0.71]]], dtype=float32)

### Some performance tests on the test set
Now, let's test it on the whole test set. The performance obtained here, should tell us a lot on how fast we can actually train the learner.

In [114]:
# let's have some more motifs (10)
kernel = np.tile(np.random.rand(4,6), [100, 2,1,1])
cRBM.setCustomKernels(kernel)

# perform forward and backward pass and calculate the gradient
start = time.time()
res = cRBM.sigmoid(cRBM.forwardBatch(dataMat))
vis = cRBM.softmax(cRBM.backwardBatch(res))
grad = cRBM.gradient(res, dataMat)
print "Forward pass: " + str(res.shape)
print "Backward pass: " + str(vis.shape)
print "Gradient: " + str(grad.shape)
print "Time for processing (for, back, grad) of " + str(dataMat.shape[0]) + " sequences: " + str((time.time()-start))

New motifs set. # Motifs: 100 K-mer-Length: 6
Forward pass: (1000, 100, 1, 145)
Backward pass: (1000, 4, 150)
Gradient: (1000, 100, 4, 6)
Time for processing (for, back, grad) of 1000 sequences: 1.9392850399


In [22]:
#del cRBM, test_set, allSeqs, pwms